In [226]:
from collections import defaultdict

In [227]:
# Bond notes below
# - Goal = Maximize yield for the entire portfolio. 

# Rules: 
# 1) Total market value on the portfolio should be $280m
# 2) Market value by tenor bucket:  
#              - 1y $90m 
#              - 5y $40m 
#              - 6y $150m 
# 3) Average rating across the portfolio and each tenor bucket should be A- or higher (7.0 or higher) 
# 4) You cannot exceed the market value cap per ticket that are in column A and B of the caps and bucket tab 
# 5) You cannot exceed the market value caps for each bond in column E and F of the caps and buckets tab"

In [228]:
total_market_value = 280

In [229]:
ticker_cap = {
    "AAPL": 10,
    "MSFT": 10,
    "ABIBB": 25,
    "LLOYDS": 7.5,
    "ADI": 7.5
}
bond_cap = {
    "AAPL": 7.5,
    "MSFT": 7.5,
    "ABIBB": 7.5,
    "LLOYDS": 5,
    "ADI": 5
}
tenor_cap = {
    1: 90,
    5: 40,
    6: 150,
}

In [230]:
class Bond:
    def __init__(self, isin, ticker, maturity_bucket, rating_score, yield_level, price_level):
        self.isin = isin 
        self.ticker = ticker 
        self.maturity_bucket = maturity_bucket 
        self.rating_score = rating_score 
        self.yield_level = yield_level 
        self.price_level = price_level

In [231]:
class Position:
    def __init__(self, size: float, bond: Bond):
        self.size = size
        self.bond = bond
        
    def market_value(self):
        return self.size * self.bond.price_level
        
    def __str__(self):
        return f"<{self.bond.ticker} | {self.size}mm>"
    
    def __repr__(self):
        return self.__str__()

In [232]:
class Portfolio:
    def __init__(self, position_list, ticker_cap, bond_cap, tenor_limits, minimum_rating):
        self.position_list = position_list
        self.ticker_cap = ticker_cap
        self.bond_cap = bond_cap
        self.tenor_limits = tenor_limits
        self.minimum_rating = minimum_rating
    
    def calculate_average_yield(self):
        yield_sum = 0.0
        notional = 0.0
        for position in self.position_list:
            yield_sum += (position.size * position.bond.yield_level)
            notional += position.size
        return yield_sum / notional
    
    def tenor_buckets(self):
        buckets = defaultdict(float)
        for position in self.position_list:
            buckets[position.bond.maturity_bucket] += position.size
        return buckets
    
    def ticker_buckets(self):
        buckets = defaultdict(float)
        for position in self.position_list:
            buckets[position.bond.ticker] += position.size
        return buckets
    
    def positions_by_tenor(self):
        buckets = defaultdict(list)
        for position in self.position_list:
            buckets[position.bond.maturity_bucket].append(position)
        return buckets
    
    def market_value(self):
        mv = 0.0
        for position in self.position_list:
            mv += position.market_value()
        return mv / 100
    
    def average_rating(self):
        rating_score_sum = 0.0
        notional = 0.0
        for position in self.position_list:
            rating_score_sum += (position.size * position.bond.rating_score)
            notional += position.size
        return rating_score_sum / notional
    
    def average_rating_by_tenor(self):
        positions_by_tenor = self.positions_by_tenor()
        buckets = defaultdict(float)
        for tenor in positions_by_tenor:
            rating_score_sum = 0.0
            notional = 0.0
            for position in positions_by_tenor[tenor]:
                rating_score_sum += (position.size * position.bond.rating_score)
                notional += position.size
            buckets[tenor] = rating_score_sum / notional
        return buckets
    
    def get_violations(self):
        violations = []
        ticker_buckets = self.ticker_buckets()
        for ticker in ticker_buckets:
            if ticker not in self.ticker_cap:
                violations.append(f"No ticker cap for ticker: {ticker}")
            elif ticker_buckets[ticker] > self.ticker_cap[ticker]:
                violations.append(f"Ticker exceeds limit: {ticker}")
        for position in self.position_list:
            if position.size > self.bond_cap[position.bond.ticker]:
                violations.append(f"Position size exceeds limit: {position.bond.ticker}")
        
        tenor_buckets = self.tenor_buckets()
        for tenor in tenor_buckets:
            if tenor not in self.tenor_limits:
                violations.append(f"Tenor not in tenor_limits: {tenor}")
            if tenor_buckets[tenor] > self.tenor_limits[tenor]:
                violations.append(f"Tenor size exceeds limit: {tenor}")
        
        average_rating = self.average_rating()
        if self.average_rating() > self.minimum_rating:
            violations.append(f"Average rating is {average_rating}, below min rating {self.minimum_rating}")
        return violations

In [233]:
b1 = Bond("12345", "AAPL", 5, 3, 3.5, 110.45)
b2 = Bond("12345", "MSFT", 5, 3, 3.7, 105.73)
p1 = Position(5, b1)
p2 = Position(5, b2)
b3 = Bond("12345", "AAPL", 5, 3, 3.4, 120.35)
p3 = Position(7, b3)
b4 = Bond("12345", "LLOYDS", 5, 5, 4.1, 103.25)
p4 = Position(6, b4)

b5 = Bond("12345", "ABIBB", 5, 4, 3.8, 101.25)
p5 = Position(20, b5)

b6 = Bond("12345", "ABIBB", 1, 4, 3.1, 100.25)
p6 = Position(20, b6)

portfolio = Portfolio([p1, p2, p3, p4, p5, p6], ticker_cap, bond_cap, tenor_cap, 7)

In [234]:
portfolio.calculate_average_yield()

3.5301587301587296

In [235]:
portfolio.tenor_buckets()

defaultdict(float, {5: 43.0, 1: 20.0})

In [236]:
portfolio.ticker_buckets()

defaultdict(float, {'AAPL': 12.0, 'MSFT': 5.0, 'LLOYDS': 6.0, 'ABIBB': 40.0})

In [237]:
portfolio.position_list

[<AAPL | 5mm>,
 <MSFT | 5mm>,
 <AAPL | 7mm>,
 <LLOYDS | 6mm>,
 <ABIBB | 20mm>,
 <ABIBB | 20mm>]

In [238]:
portfolio.market_value()

65.7285

In [239]:
portfolio.average_rating()

3.8253968253968256

In [240]:
portfolio.get_violations()

['Ticker exceeds limit: AAPL',
 'Ticker exceeds limit: ABIBB',
 'Position size exceeds limit: LLOYDS',
 'Position size exceeds limit: ABIBB',
 'Position size exceeds limit: ABIBB',
 'Tenor size exceeds limit: 5']

In [241]:
portfolio.positions_by_tenor()

defaultdict(list,
            {5: [<AAPL | 5mm>,
              <MSFT | 5mm>,
              <AAPL | 7mm>,
              <LLOYDS | 6mm>,
              <ABIBB | 20mm>],
             1: [<ABIBB | 20mm>]})

In [242]:
portfolio.average_rating_by_tenor()

defaultdict(float, {5: 3.744186046511628, 1: 4.0})

In [243]:
def optimize_portfolio(bond_list, ticker_cap, bond_cap, tenor_limits, minimum_rating):
    pass

In [ ]:
# res = minimize(
#     objective_function, #maximize yield
#     x0=10 * np.random.random(n_buyers), 
#     args=(prices,), 
#     constraints=constraint,
#     bounds=bounds,
# )